#### Generate a synthetic sales dataset for ETL Extract Lab
#### Purpose: Create a realistic dataset with 100 sales records for full and incremental extraction
#### Columns: order_id, customer_name, product, quantity, price, order_date

In [37]:
#pip install pandas faker

In [38]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

# Initialize Faker for fake data
fake = Faker()

# Create 120 sales records
data = {
    'order_id': range(1, 121),
    'order_date': [fake.date_between(start_date='-1y', end_date='today') for _ in range(120)],
    'product_name': [fake.word().capitalize() + ' ' + random.choice(['Laptop', 'Phone', 'Tablet']) for _ in range(120)],
    'price': [round(random.uniform(50, 1000), 2) for _ in range(120)],
    'customer_name': [fake.name() for _ in range(120)],
    'last_updated': [fake.date_time_between(start_date='-1y', end_date='now') for _ in range(120)]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Save to CSV in the project folder
df.to_csv('custom_data.csv', index=False)
print("Dataset saved as custom_data.csv")

Dataset saved as custom_data.csv


In [39]:
df.head()

,order_id,order_date,product_name,price,customer_name,last_updated
0,1,2024-09-23,Say Tablet,838.33,Brittany Kelly,2024-09-25 14:48:57
1,2,2024-11-25,Foreign Phone,576.50,Walter Nixon,2025-05-22 19:23:48
2,3,2024-06-29,Foreign Tablet,607.06,Michael Morrison,2025-01-02 22:33:04
3,4,2024-08-16,Black Tablet,822.40,Kelly Hall,2024-09-28 00:00:16
4,5,2025-04-28,Exactly Phone,271.35,Audrey Becker,2025-04-02 11:02:14


#### ETL Extraction Lab
**Name**: Jane Doe  
**Student ID**: 20250123  

This notebook demonstrates Full and Incremental Extraction using a sales dataset.

#### Step 1: Import Libraries
We import `pandas` to handle the CSV file and `datetime` to work with dates for incremental extraction.

#### Step 2: Load the Dataset
We load `custom_data.csv` into a pandas DataFrame to inspect its contents.

In [40]:
# Read the entire dataset
df_full = pd.read_csv('custom_data.csv')
print("Full Extraction - First 5 rows:")
print(df_full.head())

Full Extraction - First 5 rows:
   order_id  order_date    product_name   price     customer_name  \
0         1  2024-09-23      Say Tablet  838.33    Brittany Kelly   
1         2  2024-11-25   Foreign Phone  576.50      Walter Nixon   
2         3  2024-06-29  Foreign Tablet  607.06  Michael Morrison   
3         4  2024-08-16    Black Tablet  822.40        Kelly Hall   
4         5  2025-04-28   Exactly Phone  271.35     Audrey Becker   

          last_updated  
0  2024-09-25 14:48:57  
1  2025-05-22 19:23:48  
2  2025-01-02 22:33:04  
3  2024-09-28 00:00:16  
4  2025-04-02 11:02:14  


#### Step 3: Full Extraction
Full Extraction copies all data from the source (`custom_data.csv`) to a new file (`full_extract.csv`).

In [44]:
# Perform Full Extraction
df.to_csv("full_extract.csv", index=False)
print("Full Extraction completed. Saved to full_extract.csv")

Full Extraction completed. Saved to full_extract.csv


#### Step 4: Incremental Extraction
Incremental Extraction only extracts records added or updated since the last extraction. We use `last_extraction.txt` to store the last extracted date and extract records with a later `order_date`.

In [43]:
from datetime import datetime
import pandas as pd

# Read the last extraction timestamp
try:
    with open('last_extraction.txt', 'r') as file:
        timestamp_str = file.read().strip()
        if timestamp_str:  # Check if the string is not empty
            last_extract_time = datetime.fromisoformat(timestamp_str)
        else:
            last_extract_time = datetime(2000, 1, 1)  # Default if file is empty
except FileNotFoundError:
    last_extract_time = datetime(2000, 1, 1)  # Default if file doesn't exist

# Convert last_updated column to datetime
df_full = pd.read_csv('custom_data.csv')  # Ensure dataset is loaded
df_full['last_updated'] = pd.to_datetime(df_full['last_updated'])

# Filter for new or updated records
df_incremental = df_full[df_full['last_updated'] > last_extract_time]
print("Incremental Extraction - New/Updated rows:")
print(df_incremental.head())

# Update last_extraction.txt with current timestamp
with open('last_extraction.txt', 'w') as file:
    file.write(datetime.now().isoformat())

Incremental Extraction - New/Updated rows:
Empty DataFrame
Columns: [order_id, order_date, product_name, price, customer_name, last_updated]
Index: []
